In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
import torchvision.transforms as transforms

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import random_split

from collections import namedtuple, defaultdict
from datetime import datetime
from sklearn.metrics import confusion_matrix

import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import random
import seaborn as sns

In [4]:
def load_dataset(user_split = True):
    X_train = None
    y_train_medid = None
    y_train_subjid = None
    y_train_ts = None
    
    X_val = None
    y_val_medid = None
    y_val_subjid = None
    y_val_ts = None
    
    X_test = None
    y_test_medid = None
    y_test_subjid = None
    y_test_ts = None

    # user_split: Determines the creation of train/val/test set. In user_split, test/val users are never seen during the train. time_split randomly splits each user's chunks into train/val/test.
    if user_split:
        data_file_path = './data/Meditation/user_based_splits_with_timestamp.pkl'
    else:
        data_file_path = './data/Meditation/time_based_splits_with_timestamp.pkl'

    with open(data_file_path, 'rb') as f:
        all_data_splits = pickle.load(f)

        X_train = all_data_splits['train']['x']
        y_train_medid = all_data_splits['train']['y_med']
        y_train_subjid = all_data_splits['train']['y_subj']
        y_train_ts = all_data_splits['train']['y_ts']

        X_val = all_data_splits['val']['x']
        y_val_medid = all_data_splits['val']['y_med']
        y_val_subjid = all_data_splits['val']['y_subj']
        y_val_ts = all_data_splits['val']['y_ts']

        X_test = all_data_splits['test']['x']
        y_test_medid = all_data_splits['test']['y_med']
        y_test_subjid = all_data_splits['test']['y_subj']
        y_test_ts = all_data_splits['test']['y_ts']

    return X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts

In [6]:
user_split = True
X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, \
X_test, y_test_medid, y_test_subjid, y_test_ts = load_dataset(user_split)

In [7]:
X_train[0,:,:].shape

(64, 2560)

In [24]:
X_train[0,:,:].transpose().reshape(40, 64, 64).shape

(40, 64, 64)

In [27]:
X_train[0,:,:].transpose().reshape(64, 40, 64).mean(axis=1).shape

(64, 64)

In [33]:
X_train[0,:,:].transpose().reshape(64, 40, 64).mean(axis=1)[0,0]

4.3389654

In [31]:
np.mean(X_train[0,:,:].transpose().reshape(64, 40, 64)[0,:,0])

4.3389654

In [ ]:
X_train_list = []
X_val_list = []
X_test_list = []

for i in range(X_train.shape[0]):
    X_train_list.append(np.mean(X_train[i,:,:], axis=1))
X_train_list = np.array(X_train_list)
print(X_train.shape, X_train_list.shape)

for i in range(X_val.shape[0]):
    X_val_list.append(np.mean(X_val[i,:,:], axis=1))
X_val_list = np.array(X_val_list)
print(X_val.shape, X_val_list.shape)

for i in range(X_test.shape[0]):
    X_test_list.append(np.mean(X_test[i,:,:], axis=1))
X_test_list = np.array(X_test_list)
print(X_test.shape, X_test_list.shape)